In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
from numpy import newaxis
import scipy.sparse as sps
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt

In [ ]:
from pyfem.topo import Interval
from pyfem.poly import gll_points
from pyfem.sem import SEMhat
from pyfem.poly import eval_lagrange_d0 as eval_phi1d

In [ ]:
from poly import eval_P
from utils import minmod

In [ ]:
order = 1
semh = SEMhat(order)

N = 100
n_dofs = (order+1)*N

In [ ]:
L = 2.0

vertices  = np.linspace(0, L, N+1)
EtoV      = np.zeros((N, 2), dtype=np.int)
EtoV[:,0] = np.arange(N)
EtoV[:,1] = np.arange(N)+1

topo  = Interval()
xq = topo.ref_to_phys(vertices[EtoV], semh.xgll)
jacb_det = topo.calc_jacb(vertices[EtoV])[0]
dx = np.min(xq[0,1:]-xq[0,:-1])

# Restriction operator
# Removes contribution to dirch points from flux operators
R = sps.eye(n_dofs).tocsr()
R[0,0] = R[-1,-1] = 0.0

In [ ]:
# Make elem to dof map
EtoD = np.arange(N*(order+1))
EtoD = EtoD.reshape((N, -1))

dof_phys = xq.ravel()

# Averaging operator
rows = EtoD[:,[0,-1]].ravel()
cols = EtoV.ravel()
vals = np.ones_like(cols)

FtoD = sps.coo_matrix((vals, (rows, cols))).tocsr()
AVG = FtoD.dot(FtoD.T)/2.0

# Extract face DOFS
vals = np.ones(len(rows))
FD = sps.coo_matrix((vals, (rows, rows))).tocsr()
# Set face signs
vals[::2] = -1
SD = sps.coo_matrix((vals, (rows, rows))).tocsr()

# Jump operator
JUMP = FtoD.dot(SD.dot(FtoD).T)

In [ ]:
# Build Advection operator
C = sps.kron(sps.eye(N), semh.Ch).tocsr()

# Build full elemental mass matrix
x, w = topo.get_quadrature(order+1)
P = eval_phi1d(semh.xgll, x).T
G = sps.dia_matrix((w, 0), shape=(len(x), len(x)))
Bf = P.T.dot(G.dot(P))*jacb_det
Bfinv = np.linalg.inv(Bf)

# Using trick from book
V    = eval_P(order, semh.xgll).T
Vinv = np.linalg.inv(V)
Minv = V.dot(V.T)/jacb_det
Binv = sps.kron(sps.eye(N), Minv).tocsr()

In [ ]:
T  = 0.4
dt = 0.001
nt = int(round(T/dt))
T, dt, nt

## Problem Setup

In [ ]:
# # Euler -- Sod's Shock Tube

# use_filter = False

# gamma = 7.0/5.0
# def calc_flux(u):
    
#     f = np.zeros_like(u)
#     p = (gamma-1)*(u[:,2]-.5*u[:,1]**2/u[:,0])
#     f[:,0] = u[:,1]
#     f[:,1] = u[:,1]**2/u[:,0]+p
#     f[:,2] = (u[:,2]+p)*u[:,1]/u[:,0]
    
#     return f

# def calc_p(u):
#     p = (gamma-1)*(u[:,2]-.5*u[:,1]**2/u[:,0])
#     return p

# def calc_eig(u):
#     p = calc_p(u)
#     eig  = np.abs(u[:,1]/u[:,0])
#     eig += np.sqrt(gamma*p/u[:,0])
#     return eig

# # Set up classical Sod's shock tube
# u0 = np.zeros((n_dofs, 3))
# u0[:,0] = 1.0
# u0[:,0][dof_phys>=0.5] = 0.125
# u0[:,1] = 0.0
# u0[:,2] = 1.0
# u0[:,2][dof_phys>=0.5] = 0.1
# u0[:,2] /= (gamma-1)

# np.max(calc_eig(u0))

In [ ]:
# Burger -- Simple Shock

def calc_flux(u):
    
    return u*u/2.0


def calc_eig(u):

    return np.abs(u[:,0])

ul = 2.0
ur = 1.0
s  = (calc_flux(ur)-calc_flux(ul))/(ur-ul)

u0 = np.ones((n_dofs, 3))*ur
u0[dof_phys<=L/2.0,:] = ul

ue = np.ones_like(u0)*ur
ue[dof_phys<=L/2.0+s*dt*nt,:] = ul

s, s*T+L/2.0

In [ ]:
# Slope limiter

# elem midpoints
x0 = (xq[:,0]+xq[:,-1])/2.0
# elem widths (assuming uniform)
h  = (xq[:,-1]-xq[:,0])[0]

# Remove scale factors built into V
# (b.c. of the def used in nodal-dg)
nv  = np.arange(order+1)
gam = 2.0/(2.0*nv+1)
G = sps.dia_matrix((1.0/np.sqrt(gam),0),
                  shape=Vinv.shape)
G = G.dot(Vinv)
    
def slope_limit(u):
    """ MUSCL Limiter
    """
    us = u.reshape((-1,order+1)).T
    avg, slope = G.dot(us)[[0,1]]
    
    # The two comes from the domain of Legendre polys
    slope *= 2.0/h
    u = u.reshape((-1,order+1))
    
    h2 = h
    m = minmod(slope[1:-1],
               (avg[2:]-avg[1:-1])/h2,
               (avg[1:-1]-avg[:-2])/h2)
    
    # xq has shape (n_elem, n_dof_per_elem)
    # This is why the rest of the arrays need to use newaxis
    u[1:-1] = avg[1:-1,newaxis]+(xq-x0[:,newaxis])[1:-1]*m[:,newaxis]
    
def apply_limiter(u):
    slope_limit(u[:,0])
    slope_limit(u[:,1])
    slope_limit(u[:,2])
    

In [ ]:
# Integrate with RK4
u  = u0.copy()
#ue = u0.copy()
f0 = calc_flux(u0[[0,-1],:])
lambda_max = np.max(calc_eig(u0))

for k in range(nt):
        
    #lambda_max = np.max(calc_eig(u))
    #dt = .1*dx/lambda_max
    
    FLUX = AVG+(lambda_max/2.0)*JUMP
    F = Binv.dot(-C+R.dot(SD.dot(FD-FLUX)))
    g = lambda u: F.dot(calc_flux(u))
        
    k1 = g(u)
    apply_limiter(k1)
    k1[[0,-1],:] = f0
    
    k2 = g(u+(dt/2.0)*k1)
    apply_limiter(k2)
    k2[[0,-1],:] = f0
    
    k3 = g(u+(dt/2.0)*k2)
    apply_limiter(k3)
    k3[[0,-1],:] = f0
    
    k4 = g(u+dt*k3)
    apply_limiter(k4)
    k4[[0,-1],:] = f0
    
    u = u+(dt/6.0)*(k1+2*k2+2*k3+k4)
    apply_limiter(u)
    u[[0,-1],:] = u0[[0,-1],:]

    #print np.max(calc_eig(u)), np.max(calc_eig(u))*dt/dx, dt, .5*dx/np.max(calc_eig(u))

In [ ]:
plt.figure()
plt.plot(dof_phys, u[:,0])
plt.plot(dof_phys, ue[:,0], 'g--')
plt.ylabel('$\\rho$', size=16)

plt.figure()
plt.plot(dof_phys, u[:,1])
plt.plot(dof_phys, ue[:,1], 'g--')
plt.ylabel('$u$', size=16)

plt.figure()
plt.plot(dof_phys, u[:,2])
plt.plot(dof_phys, ue[:,2], 'g--')
plt.ylabel('$E$', size=16)